<a href="https://colab.research.google.com/github/somitrasingh/deeplearning/blob/main/QA_System_SimpleRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import torch
import pandas as pd
import numpy as np

In [132]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

In [133]:
df.shape

(90, 2)

In [134]:
df.head(5)

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [72]:
def tokenize(text):
  text = text.lower()
  text = text.replace("?", "")
  text = text.replace("'", "")
  return text.split()

In [95]:
vocab = {"<UNK>": 0}
def build_vocab(data):
  q = tokenize(data['question'])
  a = tokenize(data['answer'])
  text = q + a
  for token in text:
    if token not in vocab:
      vocab[token] = len(vocab)




In [96]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [97]:
len(vocab)

324

In [98]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [108]:
text = tokenize(df.iloc[0]['question'])
x = []
for token in text:
  x.append(vocab[token])

torch.tensor(x)

tensor([1, 2, 3, 4, 5, 6])

In [109]:
def text_to_indices(text, vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [100]:
import torch
from torch.utils.data import Dataset, DataLoader

In [124]:
class QARetriever(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    q = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    a = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(q), torch.tensor(a)




In [125]:
dataset = QARetriever(df, vocab)

In [126]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [145]:
for q, a in dataloader:
  print(q, a[0])

tensor([ 42, 137,   2, 138,  39, 175, 269]) tensor([99])
tensor([ 42, 137,   2, 226,  12,   3, 227, 228]) tensor([155])
tensor([ 1,  2,  3,  4,  5, 73]) tensor([74])
tensor([ 42, 137, 118,   3, 247,   5, 248]) tensor([249])
tensor([ 10,  75, 111]) tensor([112])
tensor([ 10, 140,   3, 141, 142,  12, 143,  83,   3, 144]) tensor([145])
tensor([  1,   2,   3,  17, 115,  83,  84]) tensor([116])
tensor([  1,   2,   3,  37, 133,   5,  26]) tensor([134])
tensor([ 10,  11, 189, 158, 190]) tensor([191])
tensor([42, 18,  2, 62, 63,  3, 64, 18]) tensor([65])
tensor([ 42, 125,   2,  62,  63,   3, 126, 127]) tensor([128])
tensor([  1,   2,   3,   4,   5, 279]) tensor([280])
tensor([ 1,  2,  3, 24, 25,  5, 26, 19, 27]) tensor([28])
tensor([ 10,  29, 130, 131]) tensor([132])
tensor([10, 96,  3, 97]) tensor([98])
tensor([ 42, 137,   2,  62,  39,   3, 322, 323]) tensor([6])
tensor([ 1,  2,  3, 92, 93, 94]) tensor([95])
tensor([ 10,  75, 208]) tensor([209])
tensor([ 42, 117, 118,   3, 119,  94, 120]) ten

In [136]:
import torch.nn as nn

In [171]:
class SimpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embeddings = nn.Embedding(vocab_size, 64)
    self.rnn = nn.RNN(64, 128, batch_first=True)
    self.fc = nn.Linear(128, vocab_size)

  def forward(self, x):
    y = self.embeddings(x)
    _, y = self.rnn(y)
    y = self.fc(y.squeeze(0))

    return y

In [172]:
import torch.optim as optim


In [173]:
model = SimpleRNN(len(vocab))

In [174]:
epochs = 20
learning_rate = 0.002

In [175]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [176]:
for epoch in range(epochs):
  total_loss = 0
  for q, a in dataloader:
    y = model(q)
    optimizer.zero_grad()
    loss = criterion(y, a[0])
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  avg_loss = total_loss/len(dataloader)
  print(f"Epoch: {epoch+1}, Loss: {avg_loss}")


Epoch: 1, Loss: 5.8095843156178795
Epoch: 2, Loss: 3.5442027237680223
Epoch: 3, Loss: 1.7410317815012402
Epoch: 4, Loss: 0.7275326453977161
Epoch: 5, Loss: 0.36028093592160276
Epoch: 6, Loss: 0.18661955733680063
Epoch: 7, Loss: 0.16161030350873867
Epoch: 8, Loss: 0.09987921019395192
Epoch: 9, Loss: 0.08156860305203331
Epoch: 10, Loss: 0.03970734614671932
Epoch: 11, Loss: 0.029749596967465347
Epoch: 12, Loss: 0.02420216236884395
Epoch: 13, Loss: 0.02041154220286343
Epoch: 14, Loss: 0.01752254695424603
Epoch: 15, Loss: 0.015226129442453384
Epoch: 16, Loss: 0.0133564503915194
Epoch: 17, Loss: 0.011849980242550373
Epoch: 18, Loss: 0.01055006160814729
Epoch: 19, Loss: 0.009465803785456552
Epoch: 20, Loss: 0.008530840996859803


In [183]:
model.eval()

SimpleRNN(
  (embeddings): Embedding(324, 64)
  (rnn): RNN(64, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=324, bias=True)
)

In [209]:
def predict(model, vocab,  q):
  q = text_to_indices(q, vocab)
  q = torch.tensor(q).unsqueeze(0)
  y = model(q)
  y = nn.functional.softmax(y, dim=1)
  value, index = torch.max(y, dim=1)

  return list(vocab.keys())[index]


In [210]:
print(predict(model, vocab, "what is capital of France"))

paris
